In [162]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder,normalize
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import accuracy_score,ConfusionMatrixDisplay
from sklearn import feature_selection
from scipy.stats import chi2_contingency
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,precision_recall_curve
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import pickle as pk
import numpy as np

### Preparation des données

In [163]:
data_prof = pd.read_csv('Data_Arbre.csv')
# data_debase = pd.read_csv('Patrimoine_Arbore.csv', encoding='latin1')

In [164]:
cols = ["fk_arb_etat","tronc_diam","haut_tot","fk_stadedev","age_estim", "clc_quartier", "feuillage", "fk_port", "fk_nomtech", "fk_revetement"]
data = data_prof[cols]

index = data[(data["fk_arb_etat"] == "Essouché") | (data["fk_arb_etat"] == "Non essouché")]
index = data[(data["fk_arb_etat"] == "SUPPRIMÉ") | 
             (data["fk_arb_etat"] == "ABATTU") | 
             (data["fk_arb_etat"] == "EN PLACE") | 
             (data["fk_arb_etat"] == "REMPLACÉ")].index

data.drop(index, inplace=True)

data.loc[data["fk_arb_etat"] == "Essouché", "fk_arb_etat"] = 1
data.loc[data["fk_arb_etat"] == "Non essouché", "fk_arb_etat"] = 0

# data.loc[data["fk_arb_etat"] != 1, "fk_arb_etat"] = 0
data.fk_arb_etat = data.fk_arb_etat.astype(int)
# data.fk_arb_etat.value_counts()

C:\Users\Alexandre\AppData\Local\Temp\ipykernel_20168\1887067551.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(index, inplace=True)
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_20168\1887067551.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fk_arb_etat = data.fk_arb_etat.astype(int)


In [165]:
#Encodage
#Get dummies de pandas encoder en hot one
encoder = OrdinalEncoder()
lst_col = ['fk_stadedev', 'clc_quartier', 'feuillage', 'fk_port', 'fk_nomtech', "fk_revetement"]

temp = data[lst_col]

data[lst_col] = encoder.fit_transform(temp)

C:\Users\Alexandre\AppData\Local\Temp\ipykernel_20168\992099324.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[lst_col] = encoder.fit_transform(temp)


In [166]:
X_data = data.drop(columns=["fk_arb_etat"])
Y_data = data["fk_arb_etat"]

# X_data = normalize(X_data)


X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=55, stratify=Y_data)

In [167]:
# Y_test.value_counts()

### AdaBoost

In [223]:
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train, Y_train)

AdaBoostClassifier(n_estimators=100, random_state=42)

In [224]:
pred = ada.predict(X_train)
# mlp.score(X_test, Y_test)
# pred
score = accuracy_score(Y_train, pred)
score

0.9869281045751634

In [225]:
pred = ada.predict(X_train)
mat  = confusion_matrix(Y_train, pred, normalize='true')
mat

array([[0.90909091, 0.09090909],
       [0.        , 1.        ]])

In [226]:
pred2 = ada.predict(X_test)
mat  = confusion_matrix(Y_test, pred2, normalize='true')
mat

array([[0.2       , 0.8       ],
       [0.11764706, 0.88235294]])

SMOTE

In [227]:
# sm = SMOTE(random_state=42, n_jobs=-1, sampling_strategy=0.4)
sm = SMOTE(random_state=42, n_jobs=-1)
X_smote, Y_smote = sm.fit_resample(X_train, Y_train)

c:\Users\Alexandre\anaconda3\Lib\site-packages\imblearn\over_sampling\_smote\base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [228]:
ada_smoted = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_smoted.fit(X_smote, Y_smote)

AdaBoostClassifier(n_estimators=100, random_state=42)

In [229]:
pred = ada_smoted.predict(X_smote)
mat  = confusion_matrix(Y_smote, pred, normalize='true')
mat

array([[0.98473282, 0.01526718],
       [0.        , 1.        ]])

In [230]:
pred2 = ada_smoted.predict(X_test)
mat  = confusion_matrix(Y_test, pred2, normalize='true')
mat

array([[0.2       , 0.8       ],
       [0.23529412, 0.76470588]])

### MLP

In [243]:
from sklearn.neural_network import MLPClassifier

In [244]:
mlp = MLPClassifier(hidden_layer_sizes=(20, 50), validation_fraction=0.2, learning_rate='adaptive', random_state=42, max_iter=10000)
mlp.fit(X_train, Y_train)

MLPClassifier(hidden_layer_sizes=(20, 50), learning_rate='adaptive',
              max_iter=10000, random_state=42, validation_fraction=0.2)

In [245]:
pred = mlp.predict(X_train)
# mlp.score(X_test, Y_test)
# pred
score = accuracy_score(Y_train, pred)
score

0.8562091503267973

In [246]:
pred = mlp.predict(X_train)
mat  = confusion_matrix(Y_train, pred, normalize='true')
mat

array([[0., 1.],
       [0., 1.]])

In [247]:
pred2 = mlp.predict(X_test)
mat  = confusion_matrix(Y_test, pred2, normalize='true')
mat

array([[0., 1.],
       [0., 1.]])

SMOTE

In [248]:
# sm = SMOTE(random_state=42, n_jobs=-1, sampling_strategy=0.4)
sm = SMOTE(random_state=42, n_jobs=-1)
X_smote, Y_smote = sm.fit_resample(X_train, Y_train)

c:\Users\Alexandre\anaconda3\Lib\site-packages\imblearn\over_sampling\_smote\base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [249]:
mlp_smoted = MLPClassifier(hidden_layer_sizes=(20, 50),validation_fraction=0.2, learning_rate='adaptive')
mlp_smoted.fit(X_smote, Y_smote)

c:\Users\Alexandre\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(20, 50), learning_rate='adaptive',
              validation_fraction=0.2)

In [250]:
pred = mlp.predict(X_smote)
mat  = confusion_matrix(Y_smote, pred, normalize='true')
mat

array([[0., 1.],
       [0., 1.]])

In [251]:
pred2 = mlp.predict(X_test)
mat  = confusion_matrix(Y_test, pred2, normalize='true')
mat

array([[0., 1.],
       [0., 1.]])

### Dummy

In [240]:
dum = DummyClassifier(random_state=42, strategy="most_frequent")
dum.fit(X_smote, Y_smote)

DummyClassifier(random_state=42, strategy='most_frequent')

In [241]:
pred = dum.predict(X_train)
# mlp.score(X_test, Y_test)
# pred
score = accuracy_score(Y_train, pred)
score

0.1437908496732026

In [242]:
pred = dum.predict(X_smote)
mat  = confusion_matrix(Y_smote, pred, normalize='true')
mat

array([[1., 0.],
       [1., 0.]])